In [1]:
!pip install torch
!pip install transformers
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from huggingface_hub import login

login(token="YOUR_TOKEN")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import torch
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import numpy as np

In [4]:
def sample_group(group, sample_size):
    if len(group) > sample_size:
        return group.sample(sample_size)
    else:
        return group

In [5]:
def embed(model, tokenizer, texts):
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    input_ids = encoded_input["input_ids"].to(device)
    attention_mask = encoded_input["attention_mask"].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    return embeddings

In [6]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

def tsne_plot(cls_emb, hue, style=None, filename=None, title=""):
    plt.rcParams["figure.figsize"] = (10,6)
    with plt.style.context('classic'):
        plt.title(title)
        sns.scatterplot(x=cls_emb[:, 0],
                        y=cls_emb[:, 1],
                        hue=hue,
                        palette="husl", s=80, style=style)
        plt.tight_layout()
        plt.rcParams["axes.labelsize"] = 20
        if filename is None:
            plt.show()
        else:
            plt.savefig(filename, dpi=300)
        plt.clf()

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
datasets = ["ishate", "toxigen", "ihc", "sbic", "dynahate"]
nof_samples = 300
target_groups = ["MUSLIMS", "WOMEN", "JEWS", "LGBTQ+", "BLACK PEOPLE", "WHITE PEOPLE", "IMMIGRANTS", "ASIAN", "DISEASE"]

In [9]:
import itertools
import os

# Create directories to save bert and hatebert plots.
os.makedirs("./bert", exist_ok=True)
os.makedirs("./hatebert", exist_ok=True)

# Download model and tokenizer from huggingface
repo_id = f"YOUR_USER/model-hatebert__trained-in-dynahate__seed-0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(repo_id, use_auth_token=True).to(device)
tokenizer = AutoTokenizer.from_pretrained(repo_id, use_auth_token=True)

for data_to_eval in datasets:

    # Read dataset to where evaluate the model
    df = pd.read_parquet(f"/content/drive/My Drive/datasets/{data_to_eval}_test.parquet.gzip")

    # Take a sample of `nof_samples` between Non-HS, Explicit HS, and Implicit HS
    df_sample = df.groupby('implicit_layer', group_keys=False).apply(lambda x: sample_group(x, nof_samples))
    df_sample = df_sample.reset_index(drop=False)
    indices = df_sample[
        df_sample["sanitized_target"].isin(target_groups) &
        (df_sample["label"] == 1)
    ].index
    texts = df_sample["text"].tolist()

    # Run on GPU if possible
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Get CLS embeddings for all the messages in `texts`
    embeddings = embed(model, tokenizer, texts)
    embeddings_np = np.array(embeddings)

    # Fit TSNE for visualization and project embeddings to 2 components
    tsne = TSNE(n_components=2)
    cls_emb = tsne.fit_transform(embeddings_np)

    # Make tsne plots with implicit and hateful labels
    tsne_plot(cls_emb,
              df_sample["implicit_layer"],
              filename=f"hatebert/dynahate -> {data_to_eval} - implicit labels.png",
    )
    tsne_plot(cls_emb,
              df_sample["label"].replace({0: "Non-HS", 1: "HS"}),
              filename=f"hatebert/dynahate -> {data_to_eval} - hs labels.png",
    )
    tsne_plot(cls_emb[indices, :],
              df_sample.iloc[indices].reset_index(drop=True)["sanitized_target"],
              filename=f"hatebert/dynahate -> {data_to_eval} - target labels.png",
              style=df_sample.iloc[indices].reset_index(drop=True)["implicit_layer"])

Some weights of the model checkpoint at BenjaminOcampo/model-hatebert__trained-in-dynahate__seed-0 were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<Figure size 640x480 with 0 Axes>